# propr

In [118]:
import pandas as pd
import re
from difflib import get_close_matches
df = pd.read_csv("../data/PRESCRIPTION.csv")
df_ref = pd.read_excel("../data/bdd_c.xlsx")

In [119]:
df_ref12 = pd.read_excel("../data/bdd_c.xlsx")

## remplire dose et freq 

In [120]:
# Extraire Durée : tout ce qui est après "pendant"
df["Durée"] = df["Prepost"].str.extract(r'pendant\s*(.*)$', expand=False)

# Extraire DOSE et Freq ensemble : tout avant "pendant"
pre_before = df["Prepost"].str.replace(r'\s*pendant.*$', '', regex=True)

# Extraire DOSE : tout avant le premier "par" ou "fois"
df["DOSE"] = pre_before.str.extract(r'^(\d+\s*\w+)', expand=False)

# Extraire Freq : tout le reste après la DOSE
df["Freq"] = pre_before.str.replace(r'^(\d+\s*\w+)\s*', '', regex=True)


def clean_freq(text):
    if pd.isna(text):
        return text
    
    text = str(text).strip()             # enlever espaces début/fin
    
    # Si virgule : garder uniquement après la première virgule
    if "," in text:
        text = text.split(",", 1)[1].strip()
    else:
        text = text.strip()
    
    return text

df["Freq"] = df["Freq"].apply(clean_freq)


## remplire dci en utilisant med 

In [121]:
def garder_premier_mot(m):
    return m.strip().split(" ")[0].strip() 
    

df_ref["Med1"] = df_ref["nom_commercial"].apply(garder_premier_mot)

In [122]:
df["Med"] = df["Prelibt"].apply(garder_premier_mot)

In [123]:
df_ref = df_ref.drop_duplicates(subset=['Med1'], keep='first')

In [124]:
import unidecode


df["Med"] = (
    df["Med"]
    .astype(str)
    .apply(lambda x: unidecode.unidecode(x))  # enlever les accents
    .str.upper()                              # mettre en majuscules
    .str.strip()                              # enlever les espaces autour
    .str.replace(r"[^\w\s]", "", regex=True)  # supprimer toutes ponctuations (virgules, points, etc.)
)

In [125]:
df["Med"] = df["Med"].astype(str)
df_ref["Med1"] = df_ref["Med1"].astype(str)



In [126]:
df

,ID,Date,Prelibt,CIP,Prepost,DCI,ATC,DOSE,Freq,Durée,Med
0,1,2004-04-14,BACTRIM cp Ad,3001069,"1 Comprimé(s), 1 fois / jour pendant 30 jour(s)",NaN,NaN,1 Comprimé,1 fois / jour,30 jour(s),BACTRIM
1,1,2004-04-14,KALETRA caps,3566794,"3 Capsule(s), Toutes les 12 heures pendant 30 ...",NaN,NaN,3 Capsule,Toutes les 12 heures,30 jour(s),KALETRA
2,1,2004-04-14,COMBIVIR cp enrobé,3466271,"1 Comprimé(s), Toutes les 12 heures pendant 30...",NaN,NaN,1 Comprimé,Toutes les 12 heures,30 jour(s),COMBIVIR
3,2,2013-10-01,TRUVADA 200MG/245MG CPR 30,3656563,1 comprimé par jour pendant 1 mois,NaN,NaN,1 comprimé,par jour,1 mois,TRUVADA
4,2,2013-10-01,ISENTRESS 400MG CPR 60,3830848,2 comprimés par jour pendant 1 mois,NaN,NaN,2 comprimés,par jour,1 mois,ISENTRESS
...,...,...,...,...,...,...,...,...,...,...,...
1048570,16524,2016-04-28,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",NaN,NaN,1 comprimé,1 fois par jour(s),28 jour(s),INSPRA
1048571,16524,2016-12-15,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",NaN,NaN,1 comprimé,1 fois par jour(s),28 jour(s),INSPRA
1048572,16524,2016-12-15,UVEDOSE 100 000UI/2ML AMP BUV 1,3322218,"1 ampoule, tous les 6 mois pendant 6 mois",NaN,NaN,1 ampoule,tous les 6 mois,6 mois,UVEDOSE
1048573,16524,2016-12-15,CLOPIDOGREL 75MG BIOGARAN CPR 30,3968423,"1 comprimé, par jour(s) pendant 30 jour(s)",NaN,NaN,1 comprimé,par jour(s),30 jour(s),CLOPIDOGREL


In [130]:

df12 = df.merge(
    df_ref[["Med1","dci", "atc"]],
    left_on="Med",
    right_on="Med1",
    how="left"
)

In [132]:
from unidecode import unidecode
df_ref["dci_clean"] = df_ref["dci"].astype(str).apply(lambda x: unidecode(x).upper().strip() if pd.notna(x) else x)

In [133]:
# Créer un set de dci existants dans df_ref pour vérification rapide
dci_set = set(df_ref["dci_clean"].dropna())

In [136]:
# Parcourir df12 et remplir dci si le Med correspond à un dci dans df_ref
def fill_dci(row):
    if pd.isna(row["dci"]):
        if row["Med"] in dci_set:
            return row["Med"]
    return row["dci"]

df12["dci"] = df12.apply(fill_dci, axis=1)

In [138]:
df12["DCI"]=df12["dci"]
df12["ATC"]=df12["atc"]


In [140]:
df12=df12.iloc[:,:10]

In [141]:
df12

,ID,Date,Prelibt,CIP,Prepost,DCI,ATC,DOSE,Freq,Durée
0,1,2004-04-14,BACTRIM cp Ad,3001069,"1 Comprimé(s), 1 fois / jour pendant 30 jour(s)",SULFAMÉTHOXAZOLE; TRIMÉTHOPRIME,NaN,1 Comprimé,1 fois / jour,30 jour(s)
1,1,2004-04-14,KALETRA caps,3566794,"3 Capsule(s), Toutes les 12 heures pendant 30 ...",LOPINAVIR; RITONAVIR,J05AR10,3 Capsule,Toutes les 12 heures,30 jour(s)
2,1,2004-04-14,COMBIVIR cp enrobé,3466271,"1 Comprimé(s), Toutes les 12 heures pendant 30...",LAMIVUDINE; ZIDOVUDINE,J05AR01,1 Comprimé,Toutes les 12 heures,30 jour(s)
3,2,2013-10-01,TRUVADA 200MG/245MG CPR 30,3656563,1 comprimé par jour pendant 1 mois,EMTRICITABINE; TÉNOFOVIR DISOPROXIL (FUMARATE DE),J05AR03,1 comprimé,par jour,1 mois
4,2,2013-10-01,ISENTRESS 400MG CPR 60,3830848,2 comprimés par jour pendant 1 mois,RALTÉGRAVIR POTASSIQUE,J05AJ01,2 comprimés,par jour,1 mois
...,...,...,...,...,...,...,...,...,...,...
1048570,16524,2016-04-28,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",ÉPLÉRÉNONE,C03DA04,1 comprimé,1 fois par jour(s),28 jour(s)
1048571,16524,2016-12-15,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",ÉPLÉRÉNONE,C03DA04,1 comprimé,1 fois par jour(s),28 jour(s)
1048572,16524,2016-12-15,UVEDOSE 100 000UI/2ML AMP BUV 1,3322218,"1 ampoule, tous les 6 mois pendant 6 mois",CHOLÉCALCIFÉROL,NaN,1 ampoule,tous les 6 mois,6 mois
1048573,16524,2016-12-15,CLOPIDOGREL 75MG BIOGARAN CPR 30,3968423,"1 comprimé, par jour(s) pendant 30 jour(s)",HYDROGÉNOSULFATE DE CLOPIDOGREL,B01AC04,1 comprimé,par jour(s),30 jour(s)


##  concatiner avec cip7

In [149]:
df=df12
df_ref=df_ref12

In [150]:
# Séparer les CIP par ';' et créer une ligne par CIP
df_ref= df_ref.assign(cip7=df_ref12["cip7"].str.split(";")).explode("cip7")
df_ref= df_ref.drop_duplicates(subset=['cip7'], keep='first')

df["CIP"] = df["CIP"].astype(str)
df_ref["cip7"] = df_ref["cip7"].astype(str)



In [151]:
df= df.merge(
    df_ref[["cip7", "dosage", "dosage_nature"]],
    left_on="CIP",
    right_on="cip7",
    how="left"
)


In [155]:
df_ref_small = df_ref[["cip7", "dci"]].copy()

df_ref_small["cip7"] = df_ref_small["cip7"].str.split(";")
df_ref_small = df_ref_small.explode("cip7")

df_ref_small["cip7"] = df_ref_small["cip7"].str.strip()


df_ref_small = df_ref_small.drop_duplicates(subset="cip7")


cip_to_dci = dict(zip(df_ref_small["cip7"], df_ref_small["dci"]))

df["DCI"] = df["cip7"].map(cip_to_dci).combine_first(df["DCI"])


In [156]:
df

,ID,Date,Prelibt,CIP,Prepost,DCI,ATC,DOSE,Freq,Durée,cip7,dosage,dosage_nature
0,1,2004-04-14,BACTRIM cp Ad,3001069,"1 Comprimé(s), 1 fois / jour pendant 30 jour(s)",SULFAMÉTHOXAZOLE; TRIMÉTHOPRIME,NaN,1 Comprimé,1 fois / jour,30 jour(s),3001069,400 mg; 80 mg,un comprimé
1,1,2004-04-14,KALETRA caps,3566794,"3 Capsule(s), Toutes les 12 heures pendant 30 ...",LOPINAVIR; RITONAVIR,J05AR10,3 Capsule,Toutes les 12 heures,30 jour(s),NaN,NaN,NaN
2,1,2004-04-14,COMBIVIR cp enrobé,3466271,"1 Comprimé(s), Toutes les 12 heures pendant 30...",LAMIVUDINE; ZIDOVUDINE,J05AR01,1 Comprimé,Toutes les 12 heures,30 jour(s),3466271,150 mg; 300 mg,un comprimé
3,2,2013-10-01,TRUVADA 200MG/245MG CPR 30,3656563,1 comprimé par jour pendant 1 mois,EMTRICITABINE; TÉNOFOVIR DISOPROXIL (FUMARATE DE),J05AR03,1 comprimé,par jour,1 mois,3656563,200 mg; 300 mg,un comprimé
4,2,2013-10-01,ISENTRESS 400MG CPR 60,3830848,2 comprimés par jour pendant 1 mois,RALTÉGRAVIR POTASSIQUE,J05AJ01,2 comprimés,par jour,1 mois,3830848,"434,4 mg",un comprimé
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,16524,2016-04-28,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",ÉPLÉRÉNONE,C03DA04,1 comprimé,1 fois par jour(s),28 jour(s),3665705,25 mg,un comprimé
1048571,16524,2016-12-15,INSPRA 25MG CPR 30,3665705,"1 comprimé, 1 fois par jour(s) pendant 28 jour(s)",ÉPLÉRÉNONE,C03DA04,1 comprimé,1 fois par jour(s),28 jour(s),3665705,25 mg,un comprimé
1048572,16524,2016-12-15,UVEDOSE 100 000UI/2ML AMP BUV 1,3322218,"1 ampoule, tous les 6 mois pendant 6 mois",CHOLÉCALCIFÉROL,NaN,1 ampoule,tous les 6 mois,6 mois,3322218,100 000 UI,une ampoule de 2 mL
1048573,16524,2016-12-15,CLOPIDOGREL 75MG BIOGARAN CPR 30,3968423,"1 comprimé, par jour(s) pendant 30 jour(s)",HYDROGÉNOSULFATE DE CLOPIDOGREL,B01AC04,1 comprimé,par jour(s),30 jour(s),NaN,NaN,NaN


In [ ]:
df.to_excel("prescription.xlsx",index=False)